In [1]:
import pandas as pd
import gc
from sklearn import preprocessing
import time
import numpy as np
import pickle

In [2]:
total =  pickle.load(open('../../data/raw/total_high_level.pkl','rb+'))
total = total.reset_index(drop=True)

In [3]:
#def timestamp_datetime(value):
#    format = '%Y-%m-%d %H:%M:%S'
#    value = time.localtime(value)
#    dt = time.strftime(format, value)
#    return dt
#total['context_timestamp'] = total['context_timestamp'].apply(timestamp_datetime)
#total['context_timestamp'] = pd.to_datetime(total['context_timestamp'])
#total["day"] = total["context_timestamp"].dt.day
#total["hour"] = total["context_timestamp"].dt.hour

In [4]:
def visited_shop(x):
    path = ""
    if np.isnan(x["shop_id_0"]):
        return str(-1)
    elif x["shop_id_0"] == 0:
        path += "1"
    else:
        path += "0"
        
    if np.isnan(x["shop_id_1"]):
        return path;
    elif x["shop_id_1"] == 0:
        path += "1"
    else:
        path += "0"

    if np.isnan(x["shop_id_2"]):
        return path;
    elif x["shop_id_2"] == 0:
        path += "1"
    else:
        path += "0"
        
        
    if np.isnan(x["shop_id_3"]):
        return path;
    elif x["shop_id_3"] == 0:
        path += "1"
    else:
        path += "0"

    return path

def visited_item(x):
    path = ""
    if np.isnan(x["item_id_0"]):
        return str(-1)
    elif x["item_id_0"] == 0:
        path += "1"
    else:
        path += "0"
        
    if np.isnan(x["item_id_1"]):
        return path;
    elif x["item_id_1"] == 0:
        path += "1"
    else:
        path += "0"
        
    if np.isnan(x["item_id_2"]):
        return path;
    elif x["item_id_2"] == 0:
        path += "1"
    else:
        path += "0"
    return path
def get_current_click_history(data):
    #最近两次时间差
    data.sort_values(by=['user_id', 'context_timestamp'], inplace=True)
    data['negative_seconds_gap'] = data.groupby(['user_id'])['context_timestamp'].diff(-1)
    data["negative_seconds_gap"] = data["negative_seconds_gap"].dt.seconds.map(lambda x: 86400 - x if x!=0 else 0)
    data["negative_seconds_gap"] = data["negative_seconds_gap"].fillna(-1)

    #
    data['negative_is_same_shop_id'] = data.groupby(['user_id'])['shop_id'].diff(-1)
    data["negative_is_same_shop_id"] = data["negative_is_same_shop_id"].map(lambda x:  0 if x != 0 else 1)
    data["negative_is_same_shop_id"] = data["negative_is_same_shop_id"].fillna(-1)
    
    data['negative_is_same_item_id'] = data.groupby(['user_id'])['item_id'].diff(-1)
    data["negative_is_same_item_id"] = data["negative_is_same_item_id"].map(lambda x:  0 if x != 0 else 1)
    data["negative_is_same_item_id"] = data["negative_is_same_item_id"].fillna(-1)
    
    
    #前3次访问shop_id的路径

    data['shop_id_0'] = data.groupby(['user_id'])['shop_id'].diff(-1)
    data['shop_id_1'] = data.groupby(['user_id'])['shop_id'].diff(-2)
    data['shop_id_2'] = data.groupby(['user_id'])['shop_id'].diff(-3)
    data['shop_id_3'] = data.groupby(['user_id'])['shop_id'].diff(-4)

    data["negative_visited_shop"] = data.apply(lambda x: visited_shop(x), axis=1)
    
    data.pop("shop_id_0")
    data.pop("shop_id_1")
    data.pop("shop_id_2")
    data.pop("shop_id_3")
    lbl = preprocessing.LabelEncoder()
    data["negative_visited_shop"] = lbl.fit_transform(data['negative_visited_shop'])


    #前3次访问item_id的路径
    data['item_id_0'] = data.groupby(['user_id'])['item_id'].diff(-1)
    data['item_id_1'] = data.groupby(['user_id'])['item_id'].diff(-2)
    data['item_id_2'] = data.groupby(['user_id'])['item_id'].diff(-3)

    data["negative_visited_item"] = data.apply(lambda x: visited_item(x), axis=1)
    
    data.pop("item_id_0")
    data.pop("item_id_1")
    data.pop("item_id_2")
    lbl = preprocessing.LabelEncoder()
    data["negative_visited_item"] = lbl.fit_transform(data['negative_visited_item'])

    return data

In [5]:
data = total.loc[(total["context_timestamp"] >= "20180907"),
              ["instance_id","context_timestamp","user_id","is_trade","day","shop_id","item_id"]]

In [6]:
current_click_history = pd.DataFrame()
for day in [7]:
    print(day)
    target_data = data.loc[data["day"]==day,["context_timestamp","user_id","instance_id","is_trade","shop_id","item_id"]]
    target_data = get_current_click_history(target_data)
    current_click_history = pd.concat([current_click_history,target_data])

7


In [7]:
current_click_history.pop("item_id")
current_click_history.pop("shop_id")
current_click_history.pop("user_id")
current_click_history.pop("instance_id")
current_click_history.pop("context_timestamp")
current_click_history = current_click_history.sort_index()

In [8]:
current_click_history.to_pickle('../../data/processed/leak_feature.pkl')

In [10]:
current_click_history.shape

(1597063, 6)